# Homework 2 solutions

(c) 2019 Justin Bois and Michael Elowitz. With the exception of pasted graphics, where the source is noted, this work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

This document was prepared at [Caltech](http://www.caltech.edu) with financial support from the [Donna and Benjamin M. Rosen Bioengineering Center](http://rosen.caltech.edu).

<img src="figs/caltech_rosen.png">

*This homework was generated from a Jupyter notebook.  You can download the notebook [here](hw2_solutions.ipynb). __These homework solutions may not be distributed.__*

<hr>
<br/>

In [1]:
import numpy as np
import scipy.integrate

import biocircuits

import bokeh.io
import bokeh.plotting

notebook_url = 'localhost:8890'
bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

## 2.1: Design principles for toggles, 30 pts

Consider two components, A and B, which regulate each other. A may activate or repress B, and B may activate or repress A. There are three possible architectures in this scenario, two with positive feedback and one with negative feedback, shown below.

<br />
<img src="figs/toggles.png" width="400px"/>
<br />

**a)** A circuit can behave like a toggle if it has two stable steady states, one with A high and B low and another with B high and A low. Only one of the above architectures can function as a toggle. Which one? Explain in words and sketches why _only_ the one you chose can be a toggle.

**b)** A and/or B may have ultrasensitive regulation, which we describe with a Hill coefficient, $n$, greater than one.  Show that without ultrasensitive regulation, even the architecture you chose cannot have toggle behavior.

### 2.1 solution

**a)** Let us write down differential equations for the dynamics of the concentrations of A and B, which we respectively denote by $a$ and $b$.

\begin{align}
    \frac{\mathrm{d}a}{\mathrm{d}t} &= \beta_a(b) - \gamma_a a, \\[1em]
    \frac{\mathrm{d}b}{\mathrm{d}t} &= \beta_b(a) - \gamma_b b,
\end{align}

where $\beta_a$ and $\beta_b$ are functions describing the production rate. The nullclines are then, respectively,

\begin{align}
    a &= \gamma_a^{-1}\beta_a(b), \\[1em]
    b &= \gamma_b^{-1}\beta_b(a).
\end{align}

We can rewrite the first equation as

\begin{align}
    b = \beta_a^{-1}(\gamma_a a).
\end{align}

So, the fixed points occur when the nullclines cross, or when

\begin{align}
    \beta_a^{-1}(\gamma_a a) = \gamma_b^{-1}\beta_b(a).
\end{align}

We will assume that $\beta_a$ and $\beta_b$ are monotonic functions, as is typically the case and has always been the case in class so far.  Then, by the inverse function theorem, $\beta_a^{-1}$ grows monotonically the same as $\beta_a$ does; i.e., if $\beta_a$ is monotonically increasing, so is $\beta_a^{-1}$, and if $\beta_a$ is monotonically decreasing, so is $\beta_a^{-1}$.  A necessary condition for having more than one steady state is that the functions $\beta_a^{-1}$ and $\beta_b$ are both increasing or both decreasing.  Otherwise, they could cross at exactly one point, and there would be a single steady state. For circuit (c), $\beta_a$ has a positive derivative and $\beta_b$ has a negative derivative, so this circuit cannnot have multiple steady states.  The first two may, depending on the functional forms of $\beta_a$ and $\beta_b$.

Now, say circuit (a) has two stable steady states. Since both $\beta_a$ and $\beta_b$ are monotonically _increasing_ functions, the nullclines cross when $a$ and $b$ are _both_ low or _both_ high. So, though it has two steady states, it cannot function as a toggle.

Conversely, if circuit (b) has two stable steady states, since both $\beta_a$ and $\beta_b$ are monotonically _decreasing_ functions, the nullclines cross when $a$ is high and $b$ is low, or vice versa. So, it may function as a toggle.

**b)** If the interactions are not ultrasensitive, we have, for circuit (b),

\begin{align}
    \beta_a(b) &= \frac{\beta_a^0}{1 + b/k_b}, \\[1em]
    \beta_b(a) &= \frac{\beta_b^0}{1 + a/k_a}.
\end{align}

It is easier to work with the dimensionless form of this equation, redefining $a\leftarrow a/k_a$, $b\leftarrow b/k_b$, $\beta_a \leftarrow \beta_a^0/\gamma_a k_a$, and $\beta_b \leftarrow \beta_b^0/\gamma_b k_b$.  With these redefinitions, the steady state must satisfy

\begin{align}
    \frac{\beta_a}{1+b} - a &= 0,\\[1em]
    \frac{\beta_b}{1+a} - b &= 0.
\end{align}

Solving for $b$ in terms of $a$ and substituting into the first of these equations yields

\begin{align}
    \frac{\beta_a}{1+\beta_b/(1+a)} = a.\\[1em]
\end{align}

The function on the left hand side is monotonically decreasing with $a$ (starting from a positive value at $a=0$) and that on the right hand side is monotonically increasing (starting at zero for $a = 0$). Thus, these two functions cross exactly once, so there is a single steady state. It therefore cannot function as a toggle without ultrasensitivity.

<br />

# 2.2 Autoregulation in a C1-FFL, 40 pts
*This problem is based off of problem 4.3 from Alon's book.*
The type 1 coherent feedforward loop (C1-FFL) is another motif that is found in large numbers in naturally occurring networks.  In the figure below, we show two C1-FFL networks in which the regulator Y is autoregulated.  In the C1-FFL with AND logic, Y shows autorepression, and in the C1-FFL with OR logic, it shows autoactivation.  These "decorations" on C1-FFLs often occur in nature.

<br />
<center><img src="figs/c1-ffl.png" width="600px"></center>
<br />

**a)** As we learned in lecture, the C1-FFL with AND logic shows sign-sensitive delay.  Specifically, if X is suddenly turned on, the response of Z is delayed, but if X is suddenly turned off, the response of Z is instantaneous.  Analyze the left circuit in the above figure, and compare its dynamics to the canonical C1-FFL circuit with AND logic.  Specifically address how the delay time changes. Assume that the regulation of Y follows AND logic.

**b)** The C1-FFL with OR logic also shows sign-sensitive delay. With OR logic, though, there is no delay when X is suddenly turned on, but rather delay when X is suddenly turned off.  Analyze the right circuit in the above figure and compare its dynamics to the canonical C1-FFL circuit with OR logic.  Again, address how the delay time changes, this time assuming that the regulation of Y follows OR logic.

## 2.2 solution

We begin by writing down dimensionless differential equations for the respective circuits.

\begin{align}
\text{a)}\;\;\; \gamma \dot{y} &= \beta_y\,\frac{(\kappa_x x)^{n_{xy}}}{1 + (\kappa_x x)^{n_{xy}} + (\kappa_y y)^{n_{yy}}} - y,\\[1em]
\dot{z} &= \frac{x^{n_{xz}}y^{n_{yz}}}{1 + x^{n_{xz}} y^{n_{yz}}} - z \\[1em]
\text{b)}\;\;\; \gamma \dot{y} &= \beta_y\,\frac{(\kappa_x x)^{n_{xy}}(\kappa_y y)^{n_{yy}}}{1 + (\kappa_x x)^{n_{xy}}(\kappa_y y)^{n_{yy}}}  - y, \label{eq:y_b}\\[1em]
    \dot{z} &= \frac{x^{n_{xz}} + y^{n_{yz}}}{1 + x^{n_{xz}} + y^{n_{yz}}} - z .
\end{align}

Here, $\gamma = \gamma_z/\gamma_y$ is the ratio of the decay rates of Z and Y, $\kappa_x = k_{xz} / k_{xy}$ is the ratio of the activation constants of Z and Y by X, and $\kappa_y = k_{yz} / k_{yy}$ is the ratio of the activation constants of Z and Y by Y.

To analyze the response of these circuits to steps up and down in X, I made interactive plots. Because I am comparing two circuits in one plot with lots of sliders, I custom built the plots. The code cell is large, but the tool is useful and informative.

In [2]:
# Regulation functions
def aa_and(x, y, n_x, n_y, kappa_x=1, kappa_y=1):
    return ((kappa_x*x)**n_x * (kappa_y*y)**n_y 
            / (1 + (kappa_x*x)**n_x * (kappa_y*y)**n_y))


def aa_or(x, y, n_x, n_y, kappa_x=1, kappa_y=1):
    return (((kappa_x*x)**n_x + (kappa_y*y)**n_y) 
            / (1 + (kappa_x*x)**n_x + (kappa_y*y)**n_y))


def rr_and(x, y, n_x, n_y, kappa_x=1, kappa_y=1):
    return 1 / (1 + (kappa_x*x)**n_x) / (1 + (kappa_y*y)**n_y)


def rr_or(x, y, n_x, n_y, kappa_x=1, kappa_y=1):
    return (1 + (kappa_x*x)**n_x + (kappa_y*y)**n_y) / (1 + (kappa_x*x)**n_x) / (1 + (kappa_y*y)**n_y)


def ar_and(x, y, n_x, n_y, kappa_x=1, kappa_y=1):
    return (kappa_x*x)**n_x / (1 + (kappa_x*x)**n_x + (kappa_y*y)**n_y)


def ar_or(x, y, n_x, n_y, kappa_x=1, kappa_y=1):
    return (1 + (kappa_x*x)**n_x) / (1 + (kappa_x*x)**n_x + (kappa_y*y)**n_y)


def a_hill(x, n, kappa=1):
    kxn = (kappa*x)**n
    return kxn / (1 + kxn)


def r_hill(x, n, kappa=1):
    return 1 / (1 + (kappa*x)**n)


def cffl_rhs(yz, t, beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz, 
             circuit, x):
    """
    Right hand side of ODEs for decorated C1-FFL.
    """
    # Unpack y and z concentrations
    y, z = yz

    # Make sure solver didn't step to zero
    if z < 0:
        z = 0.0
    if y < 0:
        y = 0.0

    # Compute y dynamics
    if circuit == 'a':
        dy_dt = beta_y * ar_and(x, y, n_xy, n_yy, kappa_x, kappa_y) - y
        dz_dt = aa_and(x, y, n_xz, n_yz, 1, 1) - z
    elif circuit == 'b':
        dy_dt = beta_y * aa_or(x, y, n_xy, n_yy, kappa_x, kappa_y) - y
        dz_dt = aa_or(x, y, n_xz, n_yz, 1, 1) - z
    else:
        raise RuntimeError('Invalid circuit.')

    return np.array([dy_dt/gamma, dz_dt])


def z_trace(t, beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz,
            circuit, t_0, tau, x_0):
    """
    Trace of z over time for a step up and step down.
    """
    # Compute steady state
    yz0 = np.array([0.0, 0.0])

    # Time points
    t0 = t[t<t_0]
    t1 = np.concatenate(((t_0,), t[np.logical_and(t_0<t, t<(t_0+tau))]))
    t2 = np.concatenate(((t_0+tau,), t[t>t_0+tau]))

    # Solve ODES being careful around discontinuities
    args = (beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz, circuit,
            0)
    yz_0 = scipy.integrate.odeint(cffl_rhs, yz0, t0, args=args)

    args = (beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz, circuit,
            x_0)
    yz_1 = scipy.integrate.odeint(cffl_rhs, yz_0[-1,:], t1, args=args)

    args = (beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz, circuit,
            0)
    yz_2 = scipy.integrate.odeint(cffl_rhs, yz_1[-1,:], t2, args=args)

    # Piece results together
    yz = np.vstack((yz_0[:-1,:], yz_1[:-1,:], yz_2))

    return yz[:,1]

# Parameter values
x_0 = 10
beta_y = 1
kappa_x = 1
kappa_y = 1
gamma = 1
n_xy = 2
n_xz = 2
n_yy = 2
n_yz = 2
t = np.linspace(0, 20, 400)
t_0 = 1
tau = 9

def _plot_app(doc):
    # Set up plots
    plots = [bokeh.plotting.figure(width=600, height=225,
                                    y_axis_label='norm. z', title='circuit a'),
             bokeh.plotting.figure(width=600, height=250,
                                    y_axis_label='norm. z', title='circuit b',
                                    x_axis_label='dimensionless time')]

    # Generate curves
    z_a = z_trace(t, beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz, 'a',
                  t_0, tau, x_0)
    z_b = z_trace(t, beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz, 'b',
                  t_0, tau, x_0)

    # Normalize
    z_a /= z_a.max()
    z_b /= z_b.max()

    # Build plots
    sources = [bokeh.models.ColumnDataSource(data={'t': t, 'z': z_a}),
               bokeh.models.ColumnDataSource(data={'t': t, 'z': z_b})]
    plots[0].line('t', 'z', source=sources[0], line_width=3)
    plots[1].line('t', 'z', source=sources[1], line_width=3)

    # Link ranges
    plots[1].x_range = plots[0].x_range
    plots[1].y_range = plots[0].y_range

    # Set up widgets
    x_val = bokeh.models.Slider(title='x', value=10, start=0.1, end=10, step=0.02)
    beta_y_val = bokeh.models.Slider(title='βy', value=1, start=0.1, end=10,
                                     step=0.02)
    log_kappax_val = bokeh.models.Slider(title='log10(κx)', value=0, start=-2,
                                        end=2, step=0.02)
    log_kappay_val = bokeh.models.Slider(title='log10(κy)', value=0, start=-2,
                                        end=2, step=0.02)
    log_gamma_val = bokeh.models.Slider(title='log10(γ)', value=0, start=-2,
                                        end=2, step=0.02)
    nxy_val = bokeh.models.Slider(title='nxy', value=2, start=1, end=10,
                                  step=0.02)
    nxz_val = bokeh.models.Slider(title='nxz', value=2, start=1, end=10,
                                  step=0.02)
    nyy_val = bokeh.models.Slider(title='nyy', value=2, start=1, end=10,
                                  step=0.02)
    nyz_val = bokeh.models.Slider(title='nyz', value=2, start=1, end=10,
                                  step=0.02)
    norm_z = bokeh.models.Toggle(label='normalize z', button_type='success',
                                 active=True)

    widgets = [x_val, beta_y_val, log_kappax_val, log_kappay_val, log_gamma_val, 
               nxy_val, nxz_val, nyy_val, nyz_val, norm_z]

    # Set up callbacks
    def update_data(attrname, old, new):
        # New parameter values
        x_0 = x_val.value
        beta_y = beta_y_val.value
        kappa_x = 10**log_kappax_val.value
        kappa_y = 10**log_kappay_val.value
        gamma = 10**log_gamma_val.value
        n_xy = nxy_val.value
        n_xz = nxz_val.value
        n_yy = nyy_val.value
        n_yz = nyz_val.value

        # Generate new curves
        z_a = z_trace(t, beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz,
                      'a', t_0, tau, x_0)
        z_b = z_trace(t, beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz,
                      'b', t_0, tau, x_0)

        if norm_z.active:
            z_a /= z_a.max()
            z_b /= z_b.max()

        sources[0].data = {'t': t, 'z': z_a}
        sources[1].data = {'t': t, 'z': z_b}

    def update_scale(new):
        # New parameter values
        x_0 = x_val.value
        beta_y = beta_y_val.value
        kappa_x = 10**log_kappax_val.value
        kappa_y = 10**log_kappay_val.value
        gamma = 10**log_gamma_val.value
        n_xy = nxy_val.value
        n_xz = nxz_val.value
        n_yy = nyy_val.value
        n_yz = nyz_val.value

        # Generate new curves
        z_a = z_trace(t, beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz,
                      'a', t_0, tau, x_0)
        z_b = z_trace(t, beta_y, kappa_x, kappa_y, gamma, n_xy, n_xz, n_yy, n_yz,
                      'b', t_0, tau, x_0)

        if norm_z.active:
            z_a /= z_a.max()
            z_b /= z_b.max()
            plots[0].yaxis.axis_label = 'norm. z'
            plots[1].yaxis.axis_label = 'norm. z'
        else:
            plots[0].yaxis.axis_label = 'z'
            plots[1].yaxis.axis_label = 'z'

        sources[0].data = {'t': t, 'z': z_a}
        sources[1].data = {'t': t, 'z': z_b}


    for widget in widgets[:-1]:
        widget.on_change('value', update_data)
    norm_z.on_click(update_scale)

    params = bokeh.layouts.widgetbox(*tuple(widgets))
    c = bokeh.layouts.column(plots)
    doc.add_root(bokeh.layouts.row(c, params))
    
# Make a function handler for the app
handler = bokeh.application.handlers.FunctionHandler(_plot_app)
app = bokeh.application.Application(handler)

# Show the app
bokeh.io.show(app, notebook_url=notebook_url)

The key results are summarized below.

**For circuit a**: In the limit of $\kappa_y = 0$, we no longer have autorepression of Y and we get back the canonical C1-FFL. As $\kappa_y$ grows, which means the activation constant of autorepression of Y is small, so autorepression happens readily, we get lower production of Y, which, through the AND logic, results in less Z. So, the steady state level of Z gets smaller with large $\kappa_y$. However, as we have seen in lecture, if we look at the _normalized_ level of $z$, that is the time to steady state, negative feedback _increases_ response rate. So, the decoration reduces the time delay of the circuit in its response to a sudden increase in X. This effect is most apparent when $\beta_y$ is small.

The dynamics of Y have no bearing on the absence of a delay upon removal of X, since its removal immediately eliminates production of Z with AND logic.

**For circuit b**: For the decorated C1-FFL with OR logic and autoactivation of Y, when $k_{yy}$ is zero, we recover the canonical C1-FFL. As $\beta_{y}$ grows (the effect of the autoactivation become more pronounced), the time lag in the response to turning X off grows. Furthermore, we start to see a small time lag in the response to turning X on.

## 2.3: A tri-stable cell-fate determinant circuit, 30 pts

Common myeloid precursor (CMP) cells are stem cells that can either differentiate into an erythroid (red blood cell) cell or a myeloid (bone marrow) cell. Such a binary differentiation may be regulated by a genetic circuit akin to the toggle circuit we have studied in class. However, the progenitor state of the CMP cells is also stable, which suggests tristability of the underlying circuit. In particular, the transcription factors GATA1 (which we will call X for convenience) and PU.1 (which we will call Y) are mutually repressive, which gives them the toggle-like behavior. They also are autoactivating. These interactions are summarized in the circuit below.

<br />
<center><img src="figs/tristable_toggle.png" width="200px"></center>
<br />

[Huang and coworkers (_Dev. Biol._, 2007)](https://doi.org/10.1016/j.ydbio.2007.02.036) claimed that this circuit can give tristability. Your task is to assess that claim. Model the circuit where the repression of X by Y and X's autoactivation exhibits OR logic as does repression of Y by X and Y's autoactivation. Can you come up with parameter values that show tristability?

We have not covered some of the mathematical techniques, such as linear stability analysis, that help test the stability of fixed points. However, the graphical nullcline analysis we have covered is useful and sufficient to demonstrate tristability. Think about how many fixed points are necessary to get tristability.

### 2.3 solution

We first write down the dynamical equations with OR logic.

\begin{align}
&\frac{\mathrm{d}x}{\mathrm{d}t} = \beta_x\,\frac{1 + (x/k_{xa})^{n_{xa}}}{1 + (x/k_{xa})^{n_{xa}} + (y/k_{yr})^{n_{yr}}} - \gamma_x x,\\[1em]
&\frac{\mathrm{d}y}{\mathrm{d}t} = \beta_y\,\frac{1 + (y/k_{ya})^{n_{ya}}}{1 + (x/k_{xr})^{n_{xr}} + (y/k_{ya})^{n_{ya}}} - \gamma_y y.
\end{align}

These may be nondimensionalized with

\begin{align}
&x \leftarrow x/k_{xa}, \\[1em]
&y \leftarrow x/k_{ya}, \\[1em]
&t \leftarrow \gamma_x t, \\[1em]
&\beta_x \leftarrow \frac{\beta_x}{k_{xr}\gamma_x},\\[1em]
&\beta_y \leftarrow \frac{\beta_y}{k_{yr}\gamma_y},\\[1em]
&\kappa_x = \frac{k_{xa}}{k_{xr}},\\[1em]
&\kappa_y = \frac{k_{ya}}{k_{yr}},\\[1em]
&\gamma = \gamma_y/\gamma_x.
\end{align}

The resulting dimensionless equations are

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} &= \beta_x\,\frac{1 + x^{n_{xa}}}{1 + x^{n_{xa}} + (\kappa_yy)^{n_{yr}}} - x,\\[1em]
\gamma^{-1}\frac{\mathrm{d}y}{\mathrm{d}t} &= \beta_y\,\frac{1 + y^{n_{ya}}}{1 + (\kappa_x x)^{n_{xr}} + y^{n_{ya}}} - y.
\end{align}

To find the steady states, we compute the nullclines. The $x$-nullcline, where $\mathrm{d}x/\mathrm{d}t = 0$ is given by

\begin{align}
x = \beta_x\,\frac{1 + x^{n_{xa}}}{1 + x^{n_{xa}} + (\kappa_yy)^{n_{yr}}},
\end{align}

which can be rearranged to give

\begin{align}
y^{n_{yr}} = \kappa_y^{-n_{yr}}\left(\frac{\beta_x(1+x^{n_{xa}})}{x} - 1 - x^{n_{xa}}\right).
\end{align}

We can similarly write an expression for the $y$-nullcline.

\begin{align}
x^{n_{xr}} = \kappa_x^{-n_{xr}}\left(\frac{\beta_y(1+y^{n_{ya}})}{y} - 1 - y^{n_{ya}}\right).
\end{align}

Our goal is simply to show that tristability is possible, so we will start with simplified expressions and assume that all Hill coefficients are equal, and all promoter stengths are equal. We will further assume that the Hill $k$ values for repression are equal ($k_{xr} = k_{yr}$) and that the Hill $k$ values for activation are equal ($k_{xa} = k_{ya}$). The nullclines are then

\begin{align}
&y^{n} = \kappa^{-n}\left(\frac{\beta(1+x^{n})}{x} - 1 - x^{n}\right) \\[1em]
&x^{n} = \kappa^{-n}\left(\frac{\beta(1+y^{n})}{y} - 1 - y^{n}\right).
\end{align}

We can vary these three parameters and investigate the nullclines. We need to have the nullclines cross in five places, as will soon become apparent.

To make a plot of the nullclines, we first write a function to compute the right hand side of the nullclines.

In [9]:
def nullcline(x, beta, kappa, n):
    arg = (beta*(1 + x**n) / x - 1 - x**n) / kappa**n
    res = np.empty_like(x)
    res[arg>=0] = arg[arg>=0]**(1/n)
    res[arg<0] = np.nan
    
    return res

Next, we will define parameters and make a plot. In messing around with parameters, we find that $\beta$ must not be too small nor too big, and $\kappa$ must be less than one to get five fixed points. Further, is is often the case, we need ultrasensitivity. We make a plot below.

In [18]:
beta = 5
kappa = 0.5
n = 4

# Make plot
p = bokeh.plotting.figure(height=300, width=400, 
                          x_range=[-0.2, 7], y_range=[-0.2, 7],
                          x_axis_label='x', y_axis_label='y')

# Populate nullclines
x = np.linspace(0.001, beta, 400)
p.line(nullcline(x, beta, kappa, n), x, line_width=2)
p.line(x, nullcline(x, beta, kappa, n), line_width=2, color='orange')

bokeh.io.show(p)

Consider the left most fixed point. If we move upward along the y-nullcline (orange), $x$ is decreasing. This means that the $-x$ term in the expression for $\mathrm{d}x/\mathrm{d}t$ is getting smaller in magnitude, for $\mathrm{d}x/\mathrm{d}t$ is positive. This means the system pushes rightward, toward larger $x$, and toward the fixed point. If move leftward on the x-nullcline (blue), $y$ is increasing (though just slightly). Since $y$ is increasing, $\mathrm{d}y/\mathrm{d}t$ is growing more negative, so the system pushes downward, again toward the fixed point. So, as we move away from the leftmost fixed point, the system pushes back to it, implying the leftmost fixed point is stable.

A similar analysis of the next fixed point moving rightwards reveals that the system pushes away from the fixed point, implying that it is stable. The stability alternates, from stable, unstable, stable, unstable, and finally stable, indicating tristability.